In [1]:
%pip install ultralytics roboflow opencv-python ipython

print("Libraries installed successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 138.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
Libraries installed successfully.


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=API)
project = rf.workspace("prathamesh-zyjew").project("t-bpwip")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to t-2 in yolov8:: 100%|██████████| 490/490 [00:00<00:00, 6632.81it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
import yaml
import os

dataset_path = os.path.join(os.getcwd(), 't-2')

roboflow_data_yaml_path = os.path.join(dataset_path, 'data.yaml')

with open(roboflow_data_yaml_path, 'r') as f:
    roboflow_data = yaml.safe_load(f)

class_names = roboflow_data['names']
num_classes = roboflow_data['nc']

yolov8_config = {
    'path': dataset_path,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': num_classes,
    'names': class_names
}

with open('data.yaml', 'w') as f:
    yaml.safe_dump(yolov8_config, f)

print("YOLOv8 data.yaml created successfully:")
print(yaml.dump(yolov8_config))

YOLOv8 data.yaml created successfully:
names:
- tire
nc: 1
path: /content/t-2
test: test/images
train: train/images
val: valid/images



In [4]:
import torch
from ultralytics import YOLO
import gc

def run_experiment(
    model_ckpt,
    exp_name,
    imgsz,
    lr,
    batch,
    epochs=50
):
    print(f"\n🚀 Starting {exp_name}\n")

    model = YOLO(model_ckpt)

    model.train(
        data='data.yaml',
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        optimizer='AdamW',
        lr0=lr,
        weight_decay=0.0005,
        augment=False,
        workers=2,
        name=exp_name
    )

    # 🔥 Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()

    print(f"✅ Finished {exp_name}")


In [5]:
run_experiment('yolov8n-seg.pt', 'E1_v8n_baseline', imgsz=640, lr=0.001, batch=16)



🚀 Starting E1_v8n_baseline

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E1_v8n_baseline, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, pe

In [6]:
run_experiment('yolov8n-seg.pt', 'E2_v8n_img768', imgsz=768, lr=0.001, batch=16)


🚀 Starting E2_v8n_img768

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E2_v8n_img768, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspe

In [7]:
run_experiment('yolov8n-seg.pt', 'E3_v8n_lowLR', imgsz=640, lr=0.0005, batch=16)


🚀 Starting E3_v8n_lowLR

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E3_v8n_lowLR, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspec

In [8]:
run_experiment('yolov8n-seg.pt', 'E4_v8n_batch32', imgsz=640, lr=0.001, batch=32)


🚀 Starting E4_v8n_batch32

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E4_v8n_batch32, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, pers

In [9]:
run_experiment('yolov8s-seg.pt', 'E5_v8s_compare', imgsz=640, lr=0.001, batch=16)


🚀 Starting E5_v8s_compare

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E5_v8s_compare, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, pers

In [10]:
run_experiment('yolov8m-seg.pt', 'E6_v8m_compare', imgsz=640, lr=0.001, batch=16)


🚀 Starting E6_v8m_compare

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E6_v8m_compare, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, pers

In [13]:
run_experiment('yolov8s-seg.pt','E7_v8s_img768', imgsz=768, lr=0.01, batch=16)



🚀 Starting E7_v8s_img768

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E7_v8s_img768, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspec

In [14]:
run_experiment('yolov8s-seg.pt','E8_v8s_lowLR_40ep',imgsz=640,lr=0.005,batch=16)



🚀 Starting E8_v8s_lowLR_40ep

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=E8_v8s_lowLR_40ep, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100

In [15]:
import pandas as pd
import os
#validation metrics
def print_experiment_metrics(exp_name):
    csv_path = f"runs/segment/{exp_name}/results.csv"

    if not os.path.exists(csv_path):
        print(f"❌ Results not found for {exp_name}")
        return None

    df = pd.read_csv(csv_path)

    # Use last epoch (recommended for fixed-epoch comparison)
    #last = df.iloc[-1]
    #best epoch
    best = df.loc[df['metrics/mAP50(M)'].idxmax()]

    print(f"\n Performance Metrics — {exp_name}")
    print("-" * 50)
    print(f"mAP@50 (Mask)      : {best['metrics/mAP50(M)']:.4f}")
    print(f"mAP@50-95 (Mask)   : {best['metrics/mAP50-95(M)']:.4f}")
    print(f"Precision (Mask)   : {best['metrics/precision(M)']:.4f}")
    print(f"Recall (Mask)      : {best['metrics/recall(M)']:.4f}")
    print("-" * 50)

    return {
        "Experiment": exp_name,
        "mAP50": best['metrics/mAP50(M)'],
        "mAP50-95": best['metrics/mAP50-95(M)'],
        "Precision": best['metrics/precision(M)'],
        "Recall": best['metrics/recall(M)']
    }


In [16]:
experiments = [
    "E1_v8n_baseline",
    "E2_v8n_img768",
    "E3_v8n_lowLR",
    "E4_v8n_batch32",
    "E5_v8s_compare",
    "E6_v8m_compare",
    "E7_v8s_img768",
    "E8_v8s_lowLR_40ep"
]

all_results = []

for exp in experiments:
    res = print_experiment_metrics(exp)
    if res:
        all_results.append(res)



 Performance Metrics — E1_v8n_baseline
--------------------------------------------------
mAP@50 (Mask)      : 0.9334
mAP@50-95 (Mask)   : 0.7188
Precision (Mask)   : 0.9576
Recall (Mask)      : 0.8516
--------------------------------------------------

 Performance Metrics — E2_v8n_img768
--------------------------------------------------
mAP@50 (Mask)      : 0.9535
mAP@50-95 (Mask)   : 0.7541
Precision (Mask)   : 0.9369
Recall (Mask)      : 0.8763
--------------------------------------------------

 Performance Metrics — E3_v8n_lowLR
--------------------------------------------------
mAP@50 (Mask)      : 0.9298
mAP@50-95 (Mask)   : 0.7118
Precision (Mask)   : 0.9504
Recall (Mask)      : 0.8481
--------------------------------------------------

 Performance Metrics — E4_v8n_batch32
--------------------------------------------------
mAP@50 (Mask)      : 0.9243
mAP@50-95 (Mask)   : 0.7337
Precision (Mask)   : 0.9524
Recall (Mask)      : 0.8485
-----------------------------------------